<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/siamese_network(few_shot_learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#【Few-Shot Learning (1/3): 基本概念】 https://www.bilibili.com/video/BV1V44y1r7cx/?share_source=copy_web&vd_source=cdc9fab15e0ce1d464719ce689a12b14

In [ ]:
# test

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def create_shared_network(input_shape):
    inputs = Input(shape=input_shape)

    # A few convolutional and pooling layers to extract features
    x = Conv2D(64, (10, 10), activation='relu')(inputs)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    outputs = Dense(4096, activation='sigmoid')(x)

    return Model(inputs, outputs)


In [ ]:
def siamese_network(input_shape):
    shared_network = create_shared_network(input_shape)

    # Define the two inputs
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Get feature vectors from the shared network
    encoded_a = shared_network(input_a)
    encoded_b = shared_network(input_b)

    # Compute the L1 distance between the two feature vectors
    l1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])

    # Add a dense layer with sigmoid activation to get the similarity score
    outputs = Dense(1, activation='sigmoid')(l1_distance)

    return Model(inputs=[input_a, input_b], outputs=outputs)


In [ ]:
input_shape = (105, 105, 1)  # Example input shape for Omniglot images
model = siamese_network(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 105, 105, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_6             │ (None, 105, 105, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ (None, 4096)           │     38,947,648 │ input_layer_5[0][0],   │
│                           │                        │                │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1 (Lambda)         │ (None, 4096)           │              0 │ functional_2[0][0],    │
│                           │                        │                │ functional_2[1][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │          4,097 │ lambda_1[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,951,745 (148.59 MB)

 Trainable params: 38,951,745 (148.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def generate_pairs(data, labels, num_classes):
    pairs = []
    labels_pair = []

    # Create positive and negative pairs
    for class_idx in range(num_classes):
        class_indices = np.where(labels == class_idx)[0]
        np.random.shuffle(class_indices)

        # Generate positive pairs
        for i in range(len(class_indices) - 1):
            pairs += [[data[class_indices[i]], data[class_indices[i + 1]]]]
            labels_pair += [1]

        # Generate negative pairs
        other_class = (class_idx + 1) % num_classes
        other_class_indices = np.where(labels == other_class)[0]
        pairs += [[data[class_indices[0]], data[other_class_indices[0]]]]
        labels_pair += [0]

    return np.array(pairs), np.array(labels_pair)


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()  # Example with MNIST

from tensorflow.image import resize

# Resize function for each image
def resize_images(data, new_shape=(105, 105)):
    return np.array([resize(img, new_shape).numpy() for img in data])

# Resize train and test images
X_train_resized = resize_images(X_train)
X_test_resized = resize_images(X_test)

# Update the shape after resizing
X_train_resized = np.expand_dims(X_train_resized, axis=-1)
X_test_resized = np.expand_dims(X_test_resized, axis=-1)


# # Normalize the images and reshape them for the network
# X_train = X_train.astype('float32') / 255.0
# X_train = np.expand_dims(X_train, axis=-1)


pairs, labels = generate_pairs(X_train_resized, y_train, num_classes)
history = model.fit([pairs[:, 0], pairs[:, 1]], labels, batch_size=32, epochs=10)


num_classes = len(np.unique(y_train))
pairs, labels = generate_pairs(X_train, y_train, num_classes)

# Train the Siamese network
history = model.fit([pairs[:, 0], pairs[:, 1]], labels, batch_size=32, epochs=10)

ValueError: 'images' must have either 3 or 4 dimensions.